<a href="https://colab.research.google.com/github/tammyd/CFDB_Notebook/blob/main/CFDB_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import re
from math import ceil, floor

# Load the CSV file
file_path = '/content/catfood1.csv'
df = pd.read_csv(file_path)

# Remove rows with 'discontinued' value equal to 0
df = df[df['discontinued'] == 0]

# drop columns unneeded for analysis
to_drop = ['asin', 'imageUrl', 'catfood', 'automatedUpdate', 'raw', 'baby', 'veterinary', 'discontinued']
df = df.drop(to_drop, axis=1)

df.set_index('id', inplace=True)


In [30]:
# Split out the top 5 ingredients for each cat food
ingredients_split = df['ingredients'].str.split(',', expand=True)
ingredients_split = ingredients_split.add_prefix('ing_')

none_counts = ingredients_split.isna().sum()

# Display the counts
print("Total number of rows:", ingredients_split.shape[0])
print("Count of 'None' (NaN) values in each 'ing_' column:")
print(none_counts)

df = pd.concat([df, ingredients_split], axis=1)
df = df.drop(columns=['ingredients'])

# Lets see the rows that have very few ingredients
df[df['ing_6'].isna()]

Total number of rows: 2848
Count of 'None' (NaN) values in each 'ing_' column:
ing_0         0
ing_1         0
ing_2         0
ing_3         0
ing_4         1
           ... 
ing_130    2847
ing_131    2847
ing_132    2847
ing_133    2847
ing_134    2847
Length: 135, dtype: int64


,brand,flavor,protein,fat,fibre,moisture,ash,source,updated,ing_0,...,ing_125,ing_126,ing_127,ing_128,ing_129,ing_130,ing_131,ing_132,ing_133,ing_134
id,,,,,,,,,,,,,,,,,,,,,
3279,Vital Essentials,Turkey Patties Frozen Grain Free,12.0,5.0,0.5,75.0,3.0,https://www.vitalessentialsraw.com/product/tur...,2023-01-29,Turkey,...,None,None,None,None,None,None,None,None,None,None
4631,Snappy Tom,Lites Chicken With Salmon,11.0,0.5,1.0,85.0,1.0,https://www.snappytom.com/product-page/snappy-...,2022-02-09,Chicken,...,None,None,None,None,None,None,None,None,None,None
4632,Snappy Tom,Lites Tuna,15.0,0.5,1.0,85.0,1.0,https://www.snappytom.com/product-page/snappy-...,2022-02-09,Tuna,...,None,None,None,None,None,None,None,None,None,None
4634,Snappy Tom,Lites Tuna With Pumpkin,14.0,0.5,1.0,85.0,1.0,https://www.snappytom.com/product-page/snappy-...,2022-02-09,Tuna,...,None,None,None,None,None,None,None,None,None,None
4639,Snappy Tom,Naturals Sardine Cutlet With Salmon,10.0,1.0,1.0,85.0,3.0,https://www.snappytom.com/product-page/snappy-...,2022-02-09,Sardine,...,None,None,None,None,None,None,None,None,None,None
4640,Snappy Tom,Naturals Tuna Temptations With Salmon,15.0,2.5,1.0,85.0,1.0,https://www.snappytom.com/product-page/snappy-...,2022-02-09,Tuna,...,None,None,None,None,None,None,None,None,None,None
4642,Snappy Tom,Naturals Tuna With Whitebait And Crabmeat,16.0,1.0,1.0,85.0,1.5,https://www.snappytom.com/product-page/snappy-...,2022-02-09,Tuna,...,None,None,None,None,None,None,None,None,None,None
5012,Wysong,Canine-Feline Beef,10.0,8.0,1.7,75.0,3.0,https://www.wysong.net/epigen-canned,2022-02-20,Beef,...,None,None,None,None,None,None,None,None,None,None
5013,Wysong,Canine-Feline Chicken,10.0,8.0,1.5,75.0,3.0,https://www.wysong.net/epigen-canned,2022-02-20,Chicken,...,None,None,None,None,None,None,None,None,None,None


In [31]:

# define the various types of ingredients
def get_protein_adjectives():
    return ['deboned', 'fresh deboned', 'organic', 'boneless', 'boneless/skinless', 'de-boned', 'whole', 'fresh', 'hydrolyzed', 'raw',
            'lamb', 'cutlets', 'flaked', 'shredded', 'flakes', 'freeze-dried', 'dried', 'baby', 'meat', 'dehydrated', 'fresh whole',
            'whole atlantic', 'dehydrated whole', 'fresh angus', 'fresh plains', 'fresh yorkshire', 'fresh whole pacific', 'grass-fed',
            'raw grass-fed', 'finely ground', 'king', 'ocean caught', 'humanely raised', 'pasture raised', 'wild caught', 'sustainably sourced',
            'humanely raised', 'wild pacific', 'wild atlantic', 'tongol']

def get_protein_specifics():
    return ['heart', 'thigh', 'liver', 'lung', 'liver', 'giblets', 'meal', 'white meat', 'filets', 'red meat', 'meat meal',
            'whole meat', 'cutlets', 'tripe', 'meat', '(boneless, skinless breast)', 'livers', 'gizzards', 'gizzard', 'hearts', 'necks',
            'giblets (liver, heart, kidney)', 'with ground bone', '(ground with bone)', 'kidney', 'lungs', 'trachea', 'skin',
            'with bone', 'spleen']


def get_seafood_proteins():
    return ['salmon', 'tuna', 'trout', 'mussel', 'anchovy & sardine', 'Green Lipped Mussel', 'sea bass',
               'ocean whitefish', 'herring', 'flounder', 'clam', 'pollock', 'red snapper', 'whitefish', 'cod',
               'white fish', 'mackerel', 'pacific hake', 'tilapia', 'prawns', 'yellowfin tuna', 'seabream',
               'catfish', 'sea bream', 'menhaden fish', 'ahi tuna', 'sardine', 'arctic char', 'sardines', 'basa',
               'saba', 'mackerel', 'trevally', 'barramundi', 'shrimp', 'crab', 'polluck', 'bonito', 'hoki', 'krill',
               'pilchard', 'acadian redfish', 'atlantic monkfish', 'silver hake', 'blue whiting', 'rockfish',
               'big redeye', 'barramundi', 'shirasu', 'skipjack', 'skipjack tuna', 'threadfin bream', 'red bigeye',
               'unagi', 'hake', 'kahawai']

def get_non_seafood_proteins():
    return ['goat', 'chicken', 'chicken breast', 'turkey', 'beef', 'duck', 'lamb', 'bison', 'mutton', 'rabbit',
                            'venison', 'wild boar', 'pork', 'guineafowl', 'quail', 'brushtail', 'buffalo', 'eel', 'pheasant',
                            'boar', 'kangaroo', 'alligator']


def get_by_product_phrases():
    return ["by product", "by-product"]

def get_fillers():
    return ["corn", "rice", "wheat", "starch", "flour", "soy", 'maize']

def get_fake_fillers():
    return ["wheat grass", "wheatgrass"]

def get_preservatives():
    return ['bha', 'bht', 'ethoxyquin', 'propyl gallate', 'tbhq']

def get_questionable_additives():
    return ['carrageenan', 'guar gum']

def get_sugars():
    return ['sugar', 'corn syrup']

def get_artificial_colors():
    return ['red 40', 'yellow 5', 'blue 2', 'red #3', 'red 3']

def get_all_protein_sources():
    seafood = get_seafood_proteins()
    others = get_non_seafood_proteins()
    return seafood + others

def is_protein_source(ingredient):
    if ingredient is None:  # Handle None values
      return False
    proteins = get_all_protein_sources()
    return is_ingredient_a_source(proteins, ingredient)

def is_questionable_additive(ingredient):
    if ingredient is None:  # Handle None values
      return False
    return ingredient in get_questionable_additives()

def is_artificial_color(ingredient):
    if ingredient is None:  # Handle None values
      return False
    return ingredient in get_artificial_colors()

def is_sugar(ingredient):
    if ingredient is None:  # Handle None values
      return False
    return ingredient in get_sugars()

def is_byproduct(ingredient):
    if ingredient is None:  # Handle None values
      return False

    phrases = get_by_product_phrases()
    ingredient = ingredient.strip().lower()
    for phrase in phrases:
        if phrase in ingredient:
            return True
    return False

def is_filler(ingredient):
    if ingredient is None:  # Handle None values
        return False
    phrases = get_fillers()
    fakes = get_fake_fillers()

    ingredient_lower = ingredient.lower()
    for phrase in phrases:
        if phrase in ingredient_lower:
            if not contains_any(ingredient_lower, fakes, case_sensitive=False):
                return True
    return False

def is_undesirable_preservative(ingredient):
    if ingredient is None:  # Handle None values
        return False
    phrases = get_preservatives()
    ingredient_lower = ingredient.lower()
    for phrase in phrases:
        if phrase in ingredient_lower:
            return True
    return False

def get_protein_adjectives():
    return ['deboned', 'fresh deboned', 'organic', 'boneless', 'boneless/skinless', 'de-boned', 'whole', 'fresh', 'hydrolyzed', 'raw',
            'lamb', 'cutlets', 'flaked', 'shredded', 'flakes', 'freeze-dried', 'dried', 'baby', 'meat', 'dehydrated', 'fresh whole',
            'whole atlantic', 'dehydrated whole', 'fresh angus', 'fresh plains', 'fresh yorkshire', 'fresh whole pacific', 'grass-fed',
            'raw grass-fed', 'finely ground', 'king', 'ocean caught', 'humanely raised', 'pasture raised', 'wild caught', 'sustainably sourced',
            'humanely raised', 'wild pacific', 'wild atlantic', 'tongol']

def get_protein_specifics():
    return ['heart', 'thigh', 'liver', 'lung', 'liver', 'giblets', 'meal', 'white meat', 'filets', 'red meat', 'meat meal',
            'whole meat', 'cutlets', 'tripe', 'meat', '(boneless, skinless breast)', 'livers', 'gizzards', 'gizzard', 'hearts', 'necks',
            'giblets (liver, heart, kidney)', 'with ground bone', '(ground with bone)', 'kidney', 'lungs', 'trachea', 'skin',
            'with bone', 'spleen']

def strip_brackets(text):
    if isinstance(text, str): # check if text is a string
        return re.sub(r'\[.*?\]', '', text)
    else:
        return text # return original value if not a string

def remove_non_printable(text):
    if isinstance(text, str): # check if text is a string
        return ''.join(char for char in text if char.isprintable())
    else:
        return text # return original value if not a string

    return ''.join(char for char in text if char.isprintable())

def contains_any(text, phrases, case_sensitive=True):
    if not case_sensitive:
        text = text.lower()
        phrases = [phrase.lower() for phrase in phrases]
    return any(phrase in text for phrase in phrases)

def is_ingredient_a_source(proteins, ingredient):
    if ingredient is None:  # Handle None values
        return False
    adjectives = get_protein_adjectives()
    suffixes = get_protein_specifics()

    ingredient = strip_brackets(ingredient)
    ingredient = remove_non_printable(ingredient)

    for protein in proteins:
        if isinstance(protein, str):
          if protein in ingredient:
              phrases = [protein]
              for adjective in adjectives:
                  phrases.append(f"{adjective} {protein}")
                  for suffix in suffixes:
                      phrases.append(f"{adjective} {protein} {suffix}")
              for suffix in suffixes:
                  phrases.append(f"{protein} {suffix}")
              for phrase in phrases:
                  if ingredient.strip() == phrase.strip():
                      return True

    return False

def get_ingredient_score(row):

  is_top_protein = is_protein_source(row['ing_1'])
  is_second_ing_protein = is_protein_source(row['ing_2'])
  is_third_ing_protein = is_protein_source(row['ing_3'])
  is_fourth_ing_protein = is_protein_source(row['ing_4'])
  is_fifth_ing_protein = is_protein_source(row['ing_5'])

  is_top_filler = is_filler(row['ing_1'])
  is_top_byproduct = is_byproduct(row['ing_1'])

  num_fillers = 0
  num_proteins = 0
  num_byproducts = 0
  num_undesirable_perservatives = 0
  num_questionable_additives = 0

  num_2_5_proteins = 0
  num_2_5_byproducts = 0
  num_2_5_fillers = 0
  num_2_5_perservatives = 0

  round_up = True

  for column in row.index:
        if column.startswith('ing_') and pd.notna(row[column]):
            column_index = int(column.split('_')[1])
            ingredient = row[column].strip()
            if is_protein_source(ingredient):
                num_proteins += 1
                if column_index < 5:
                  num_2_5_proteins += 1
            if is_byproduct(ingredient):
                num_byproducts += 1
                if column_index < 5:
                  num_2_5_byproducts += 1
            if is_filler(ingredient):
                num_fillers += 1
                if column_index < 5:
                  num_2_5_fillers += 1
            if is_undesirable_preservative(ingredient):
                num_undesirable_perservatives += 1
                if column_index < 5:
                  num_2_5_perservatives += 1
            if is_questionable_additive(ingredient):
                num_questionable_additives += 1
                round_up = False

  score = 2 # start with a score of 2
  if is_top_protein:
    score += 1 # add 1 if the first ingredient is a protein
  score += num_2_5_proteins * 0.5 # add 0.5 for each protein in position 2-5
  if is_top_filler or is_top_byproduct:
    score -= 1 # subtract 1 if the first ingredient is a filler or byproduct
  score -= (0.5 * num_2_5_byproducts) # -.5 for each by-product in pos 2-5

  if num_fillers > (num_2_5_fillers + (1 if is_top_filler else 0)):
    score += 0.25  # has by products outside of top 5

  if num_undesirable_perservatives > 0:
    score -= 1 # subtract 1 if undesirable preservatives

  if num_questionable_additives > 0:
    score -= 0.5 # subtract 0.5 if questionable additives

  score = max(1, score)
  if round_up:
    score = ceil(score)
  else:
    score = floor(score)

  score = min(5, score) # max score is 5

  return score



In [32]:
df['ing_score'] = df.apply(get_ingredient_score, axis=1)
df.head()

,brand,flavor,protein,fat,fibre,moisture,ash,source,updated,ing_0,...,ing_126,ing_127,ing_128,ing_129,ing_130,ing_131,ing_132,ing_133,ing_134,ing_score
id,,,,,,,,,,,,,,,,,,,,,
9,9Lives,Tender Morsels With Real Beef In Sauce,8.0,2.5,1.0,82.0,3.0,https://www.9lives.com/products/tender-morsels...,2023-01-06,Water Sufficient For Processing,...,None,None,None,None,None,None,None,None,None,1
14,9Lives,Meaty Paté With Real Ocean Whitefish,9.0,4.5,1.0,78.0,4.0,https://www.9lives.com/products/meaty-pate/mea...,2023-01-06,Meat By-products,...,None,None,None,None,None,None,None,None,None,2
17,9Lives,Meaty Paté With Real Chicken & Tuna,9.0,4.5,1.0,78.0,3.5,https://www.9lives.com/products/meaty-pate/mea...,2023-01-06,Water Sufficient For Processing,...,None,None,None,None,None,None,None,None,None,1
19,9Lives,Meaty Paté With Real Chicken,9.0,4.5,1.0,78.0,4.0,https://www.9lives.com/products/meaty-pate/mea...,2023-01-06,Meat By-products,...,None,None,None,None,None,None,None,None,None,2
20,9Lives,Meaty Paté With Real Beef,9.0,4.5,1.0,78.0,3.5,https://www.9lives.com/products/meaty-pate/mea...,2023-01-06,Meat By-products,...,None,None,None,None,None,None,None,None,None,2
